<a href="https://colab.research.google.com/github/hanjiadong0/thesis_agent/blob/main/RL_assistant_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Reasoning**:
Provide the complete and simplified Python code in a single code block as requested by the subtask.



In [36]:
!pip install openai stable-baselines3[extra] gymnasium langgraph pydantic numpy torch

In [80]:
# Combined Code for Thesis RL Agent (Ready for Merge)

# --- Imports ---
import gymnasium as gym
import torch
import torch.nn as nn
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.torch_layers import BaseFeaturesExtractor
import numpy as np
from typing import Dict, Any, List, Optional
from pydantic import BaseModel, Field
import random
import os
from langgraph.graph import StateGraph, END
from openai import OpenAI # Import OpenAI
from google.colab import drive # Import drive
import time # Import time for potential delay

# --- Mount Google Drive ---
# This needs to be done to access files in your Drive
print("Mounting Google Drive...")
try:
    # Check if already mounted
    if not os.path.exists('/content/drive'):
        drive.mount('/content/drive')
        print("Google Drive mounted successfully.")
    else:
        print("Google Drive already mounted.")
except Exception as e:
    print(f"Error mounting Google Drive: {e}. Model persistence will not work.")



Mounting Google Drive...
Google Drive already mounted.


**Reasoning**:
Analyze the `compute_reward` function to understand how it incentives state changes.



In [78]:
# --- Core RL Definitions ---

# 1. Define the state of the LangGraph using a Pydantic model
class GraphState(BaseModel):
    """Represents the minimal state of the LangGraph for the Thesis Assistant."""
    user_input: str = ""
    request_type: str = ""
    thesis_stage: str = "unknown"
    context: Dict[str, Any] = Field(default_factory=dict)
    module_response: Any = None
    rl_recommended_action: int = -1

    # Core latent state - these represent the simplified state of the student's progress/well-being
    personal_value: float = 0.5 # Represents personal growth/well-being (0.0 to 1.0)
    quality: float = 0.5 # Represents thesis quality (0.0 to 1.0)
    quantity: float = 0.0 # Represents thesis progress/quantity (0.0 to 1.0, 1.0 being completion)

    # Relevant fields based on schemas.py and potentially derivable from ai_service.py context
    thesis_field: str = "Computer Science"
    procrastination_level: str = "medium"
    writing_style: str = "neutral"
    days_to_deadline: int = 0 # Calculated from thesis_deadline

    # Simplified state variables that might be updated by "simulate" logic
    emotion_state: float = 0.5
    deadline_ratio: float = 0.0 # Kept: Derived from days_to_deadline/initial deadline

    # Add a key for the final formatted response
    formatted_response: Dict[str, Any] = Field(default_factory=dict)


# 2. Define a mock Settings class based on config.py
class MockSettings:
    """Mock Settings class mimicking config.py for AI model name access."""
    def __init__(self):
        self.AI_MODEL: str = "gpt-4o-mini" # Using a suitable model for this task

# Instantiate the mock settings
settings = MockSettings()


# 3. Define the encode_minimal_state function
RAW_STATE_DIM = 7 # Reduced dimension based on simplified GraphState
observation_space_raw = gym.spaces.Box(low=0.0, high=1.0, shape=(RAW_STATE_DIM,), dtype=np.float32)

def encode_minimal_state(state: GraphState) -> List[float]:
    """
    Encode a minimal state vector for the PPO agent using simplified GraphState.
    Returns a list of RAW_STATE_DIM float values.
    """
    state_vector = []

    # Features from simplified GraphState
    state_vector.append(state.personal_value) # 0-1
    state_vector.append(state.quality) # 0-1
    state_vector.append(state.quantity) # 0-1
    state_vector.append(state.emotion_state) # 0-1
    state_vector.append(state.deadline_ratio) # 0-1

    # Encode categorical features (simple numerical mapping)
    thesis_field_map = {"Computer Science": 0.0, "Psychology": 0.5, "History": 1.0}
    procrastination_map = {"low": 0.0, "medium": 0.5, "high": 1.0}

    state_vector.append(thesis_field_map.get(state.thesis_field, 0.5))
    state_vector.append(procrastination_map.get(state.procrastination_level, 0.5))

    # Ensure the vector matches RAW_STATE_DIM
    while len(state_vector) < RAW_STATE_DIM:
        state_vector.append(0.5) # Pad with a neutral value if needed
    while len(state_vector) > RAW_STATE_DIM:
        state_vector.pop() # Trim if needed

    return state_vector


LATENT_STATE_DIM = 3 # The target dimension for the encoder output

# Discrete action space for the 5 advisor actions:
ACTION_LABELS = ["Plan next steps", "Suggest writing", "Recommend reflection", "Offer encouragement", "Advisor feedback reminder"]
action_space = gym.spaces.Discrete(len(ACTION_LABELS))


# 4. Define the Custom State Encoder Layer (PyTorch Module)
class StateEncoder(BaseFeaturesExtractor):
    """
    Custom feature extractor (encoder) to map the raw state to the 3D latent state.
    """
    def __init__(self, observation_space: gym.spaces.Box, features_dim: int = LATENT_STATE_DIM, hidden_dim: int = 64):
        super(StateEncoder, self).__init__(observation_space, features_dim)
        self.shared = nn.Linear(self._observation_space.shape[0], hidden_dim)
        self.latent = nn.Linear(hidden_dim, features_dim)

    def forward(self, observations: torch.Tensor) -> torch.Tensor:
        observations = observations.float()
        h = torch.relu(self.shared(observations))
        return torch.sigmoid(self.latent(h))


# 5. Define the Reward Function
def compute_reward(state: GraphState, previous_state: GraphState) -> float:
    """
    Computes the reward based on the current and previous state.
    Reward is shaped by personal_value, quality, and quantity, with penalties.
    """
    personal_value = state.personal_value
    quality = state.quality
    quantity = state.quantity
    deadline_ratio = state.deadline_ratio
    time_left = 1.0 - deadline_ratio

    momentum_bonus = max(0.0, quantity - previous_state.quantity) * 0.5

    reward = (
        + 1.0 * (personal_value - 0.5)
        + 0.3 * quality
        + 0.3 * quantity
        - 0.2 * abs(quality - quantity)
        - 0.4 * (1 - time_left)
        + momentum_bonus
    )
    return reward


# 6. Define a Minimal RL Environment (Gym-style)
class ThesisEnv(gym.Env):
    """
    A minimal, simulated Gym-style RL environment for the Thesis Assistant.
    Simulates student responses and updates the raw state and GraphState.
    Accepts an optional llm_client.
    """
    def __init__(self, llm_client=None): # Accept optional llm_client
        super().__init__()
        self.observation_space = observation_space_raw
        self.action_space = action_space
        self._state = GraphState()
        self._previous_state = GraphState()
        self._raw_state = np.array(encode_minimal_state(self._state), dtype=np.float32)
        self._timestep = 0
        self._max_timesteps = 100
        self._initial_days_to_deadline = 0
        self.llm_client = llm_client # Store the provided LLM client

    def step(self, action):
        self._timestep += 1
        self._previous_state = self._state.model_copy(deep=True)

        # Simulate changes to graph state based on action and random chance (simplified)
        personal_value = self._state.personal_value
        quality = self._state.quality
        quantity = self._state.quantity
        emotion_state = self._state.emotion_state
        days_to_deadline = self._state.days_to_deadline

        if action == 0: # Plan
             quantity = min(quantity + np.random.uniform(0.01, 0.05), 1.0)
             personal_value = min(personal_value + np.random.uniform(0.01, 0.03), 1.0)
        elif action == 1: # Write
             quantity = min(quantity + np.random.uniform(0.02, 0.1), 1.0)
             quality = min(quality + np.random.uniform(0.00, 0.02), 1.0)
        elif action == 2: # Reflect
             quality = min(quality + np.random.uniform(0.01, 0.05), 1.0)
             personal_value = min(personal_value + np.random.uniform(0.01, 0.05), 1.0)
        elif action == 3: # Encourage
             emotion_state = max(emotion_state - np.random.uniform(0.1, 0.3), 0.0)
             personal_value = min(personal_value + np.random.uniform(0.02, 0.05), 1.0)
        elif action == 4: # Feedback Reminder
             quality = min(quality + np.random.uniform(0.00, 0.01), 1.0)
             emotion_state = np.clip(emotion_state + np.random.normal(0, 0.03), 0.0, 1.0)
        else:
            print(f"Warning: Unknown action index {action}. No specific simulation rule applied.")

        # Simulate time passing and update days_to_deadline and deadline_ratio
        days_passed = random.randint(1, 7)
        days_to_deadline = max(0, days_to_deadline - days_passed)
        initial_days = self._state.context.get('initial_days_to_deadline', 1)
        deadline_ratio = 1.0 - (days_to_deadline / initial_days)
        deadline_ratio = np.clip(deadline_ratio, 0.0, 1.0)

        # Update the state
        self._state.personal_value = personal_value
        self._state.quality = quality
        self._state.quantity = quantity
        self._state.emotion_state = emotion_state
        self._state.days_to_deadline = days_to_deadline
        self._state.deadline_ratio = deadline_ratio

        # Re-encode raw state
        self._raw_state = np.array(encode_minimal_state(self._state), dtype=np.float32)

        reward = compute_reward(self._state, self._previous_state)

        # Exit conditions based on simplified state
        done = self._state.quantity >= 1.0 or self._state.personal_value >= 0.95 or self._timestep >= self._max_timesteps or self._state.days_to_deadline <= 0

        info = {
            "current_graph_state": self._state.model_dump(),
            "previous_graph_state": self._previous_state.model_dump(),
            "timestep": self._timestep
            }
        # Include the llm_client in the info dictionary so the node can access it
        info['llm_client'] = self.llm_client


        return self._raw_state, reward, done, False, info

    def reset(self, seed=None, options=None):
        super().reset(seed=seed)

        initial_days = random.randint(100, 500)
        self._initial_days_to_deadline = initial_days

        self._state = GraphState(
            personal_value=np.random.uniform(0.4, 0.6),
            quality=np.random.uniform(0.4, 0.6),
            quantity=0.0,
            emotion_state=np.random.uniform(0.4, 0.6),
            deadline_ratio=0.0,
            thesis_field=random.choice(["Computer Science", "Psychology", "History"]),
            procrastination_level=random.choice(["low", "medium", "high"]),
            writing_style=random.choice(["concise", "descriptive", "neutral"]),
            days_to_deadline=initial_days
        )

        self._previous_state = self._state.model_copy(deep=True)
        self._raw_state = np.array(encode_minimal_state(self._state), dtype=np.float32)
        self._timestep = 0
        info = {"initial_graph_state": self._state.model_dump()}
        info['initial_graph_state']['context']['initial_days_to_deadline'] = self._initial_days_to_deadline
        # Also include the llm_client in the initial info
        info['llm_client'] = self.llm_client


        return self._raw_state, info

    def render(self):
        print("\nCurrent State (Simulated Environment):")
        print(f"  Raw State (partial): {self._raw_state[:5]}...")
        print(f"  Latent State: PV={self._state.personal_value:.2f}, QL={self._state.quality:.2f}, QT={self._state.quantity:.2f}")
        print(f"  Emotion: {self._state.emotion_state:.2f}, Deadline: {self._state.deadline_ratio:.2f}, Days Left: {self._state.days_to_deadline}")
        print(f"  Thesis Context: Field={self._state.thesis_field}, Procrastination={self._state.procrastination_level}, Writing Style={self._state.writing_style}")

    def close(self):
        pass

# 7. Define MODEL_PATH and policy_kwargs
# Define the base path for saving/loading from Google Drive
DRIVE_MODEL_DIR = "/content/drive/MyDrive/Colab_Thesis_Models"
# IMPORTANT: Define the filename WITH the .zip extension
MODEL_FILENAME = "thesis_policy.pt.zip" # Corrected filename
MODEL_PATH = os.path.join(DRIVE_MODEL_DIR, MODEL_FILENAME)


policy_kwargs = dict(
    features_extractor_class=StateEncoder,
    features_extractor_kwargs=dict(features_dim=LATENT_STATE_DIM, hidden_dim=64),
    net_arch=dict(pi=[64, 64], vf=[64, 64])
)

# --- Mock OpenAI Client Definition ---
class MockOpenAIClient:
    """A simple mock client to simulate OpenAI API calls during training."""
    def chat(self):
        class Completions:
            def create(self, model, messages, max_tokens=None, temperature=None):
                # Simulate a simple, consistent response for training
                class MockChoice:
                    class MockMessage:
                        content = "Mock advisor suggestion for training."
                    message = MockMessage()
                class MockResponse:
                    choices = [MockChoice()]
                return MockResponse()
        completions = Completions()
        return completions

# Define a function to create the environment, injecting a specific LLM client
def make_env_with_client(llm_client_instance):
    def _init():
        return ThesisEnv(llm_client=llm_client_instance)
    return _init
# --- End Mock OpenAI Client Definition ---


# 8. Define the LangGraph Node Functions
# Modify policy_node to use a pre-loaded model passed via config
def policy_node(state: GraphState, config: Dict[str, Any]) -> Dict[str, Any]:
    """
    Uses a pre-loaded PPO model from config to determine the action.
    Returns a dictionary update for the state with the recommended action index.
    """
    print("\n--- Executing Policy Node ---")

    # Attempt to get the pre-loaded model from the config dictionary
    # Access it via config.get("configurable", {}).get("ppo_model")
    # We now expect the model to be passed here after being loaded once before the stream
    model = config.get("configurable", {}).get("ppo_model")

    if model is None:
        print("Error: PPO model not available in config. Falling back to random action.")
        # We might get here if the model failed to load BEFORE the stream started.
        # In this case, we can optionally try to load it again or just fallback.
        # Let's stick to falling back here if it wasn't successfully loaded before.
        recommended_action = random.randint(0, len(ACTION_LABELS) - 1) # Fallback to random action
    else:
        try:
            print("PPO model found in config. Proceeding with prediction.") # Debug print
            raw_state_vector = np.array(encode_minimal_state(state), dtype=np.float32)

            if raw_state_vector.shape[0] != RAW_STATE_DIM:
                 print(f"Warning: Raw state dimension mismatch. Expected {RAW_STATE_DIM}, got {raw_state_vector.shape[0]}. Adjusting.")
                 if raw_state_vector.shape[0] > RAW_STATE_DIM:
                      raw_state_vector = raw_state_vector[:RAW_STATE_DIM]
                 else:
                      raw_state_vector = np.pad(raw_state_vector, (0, RAW_STATE_DIM - raw_state_vector.shape[0]), 'constant', constant_values=0.5)


            # Reshape observation for prediction
            obs_batch = np.array([raw_state_vector], dtype=np.float32)
            action, _states = model.predict(obs_batch, deterministic=True)
            recommended_action = int(action[0])

        except Exception as e:
            print(f"Error during model prediction: {e}. Falling back to random action.")
            recommended_action = random.randint(0, len(ACTION_LABELS) - 1) # Fallback to random action


    print(f"Policy Node Output: Recommended Action Index = {recommended_action} ({ACTION_LABELS[recommended_action]})")

    updated_context = state.context.copy()
    # Don't store raw_state in context unless needed elsewhere
    # updated_context["raw_state"] = raw_state_vector.tolist()

    # Pass the llm_client through the context if it exists
    llm_client_instance = state.context.get("llm_client")
    if llm_client_instance is not None:
        updated_context['llm_client'] = llm_client_instance


    return {"rl_recommended_action": recommended_action, "context": updated_context}


# llm_node uses the client from the environment instance (passed via context)
def llm_node(state: GraphState) -> Dict[str, Any]:
    """
    Uses the LLM client from the environment instance (passed via context)
    or falls back to the global client to generate an advisor suggestion.
    Returns a dictionary update for the state with the module response.
    """
    print("\n--- Executing LLM Abstraction Node (using OpenAI) ---")
    action_index = state.rl_recommended_action

    if action_index is None or action_index < 0 or action_index >= len(ACTION_LABELS):
        print(f"Warning: Invalid action index received: {action_index}. Defaulting to 'Plan next steps'.")
        action_index = 0

    recommended_action_label = ACTION_LABELS[action_index]

    # Attempt to get the LLM client from state context
    llm_client_instance = state.context.get("llm_client")

    if llm_client_instance is None:
        # If no client is in context, use the global client (for inference)
        global client
        llm_client_instance = client
        print("Using global OpenAI client for LLM node.")
    else:
        print("Using client from state context for LLM node (likely mock).")

    if llm_client_instance is None or not hasattr(llm_client_instance, 'chat'):
         print("Error: LLM client is not initialized or not accessible.")
         advisor_suggestion = f"Advisor Suggestion (Fallback): Consider action '{recommended_action_label}'. (Error: LLM client not initialized)"
         return {"module_response": advisor_suggestion}

    # Construct the prompt for the OpenAI API
    prompt = f"""
    You are an AI thesis advisor. Your goal is to provide concise, helpful suggestions to a student based on their current progress and a recommended action.

    Student Context:
    - Thesis Field: {state.thesis_field}
    - Procrastination Level: {state.procrastination_level}
    - Writing Style: {state.writing_style}
    - Estimated Days to Deadline: {state.days_to_deadline}
    - Current Progress (Quantity): {state.quantity:.1f}/1.0
    - Estimated Quality: {state.quality:.1f}/1.0
    - Personal Well-being: {state.personal_value:.1f}/1.0
    - Emotional State: {state.emotion_state:.1f}/1.0

    Recommended Action (from AI Agent): "{recommended_action_label}"

    Based on the recommended action and the student's context, provide a brief, encouraging, and actionable advisor suggestion (1-3 sentences). Tailor the suggestion to the student's context if relevant.
    """

    try:
        response = llm_client_instance.chat.completions.create(
            model=settings.AI_MODEL,
            messages=[
                {"role": "system", "content": "You are an AI thesis advisor."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=100,
            temperature=0.7,
        )

        if response.choices and response.choices[0].message.content:
            advisor_suggestion = response.choices[0].message.content.strip()
        else:
            advisor_suggestion = f"Advisor Suggestion: Consider action '{recommended_action_label}'. (No specific suggestion generated by LLM.)"

    except Exception as e:
        print(f"Error during LLM API call: {e}")
        advisor_suggestion = f"Advisor Suggestion (API Error): Consider action '{recommended_action_label}'. (API Error: {e})"

    return {"module_response": advisor_suggestion}


def core_update_node(state: GraphState) -> Dict[str, Any]:
    """
    Updates the core latent state and increments step_count.
    Returns a dictionary with the updates to the state.
    Assumes update_mode is "simulate".
    Simplified state updates.
    """
    update_mode = "simulate"
    print(f"\n--- Executing Core Update Node in '{update_mode}' mode ---")

    updated_context = state.context.copy()
    step_count = updated_context.get('step_count', 0) + 1
    updated_context['step_count'] = step_count

    # Get current state values
    personal_value = state.personal_value
    quality = state.quality
    quantity = state.quantity
    emotion_state = state.emotion_state
    deadline_ratio = state.deadline_ratio
    days_to_deadline = state.days_to_deadline
    action_index = state.rl_recommended_action

    if update_mode == "simulate":
        # Simulate state changes based on action
        if action_index == 0: # Plan
             quantity = min(quantity + np.random.uniform(0.01, 0.05), 1.0)
             personal_value = min(personal_value + np.random.uniform(0.01, 0.03), 1.0)
        elif action_index == 1: # Write
             quantity = min(quantity + np.random.uniform(0.02, 0.1), 1.0)
             quality = min(quality + np.random.uniform(0.00, 0.02), 1.0)
        elif action_index == 2: # Reflect
             quality = min(quality + np.random.uniform(0.01, 0.05), 1.0)
             personal_value = min(personal_value + np.random.uniform(0.01, 0.05), 1.0)
        elif action_index == 3: # Encourage
             emotion_state = max(emotion_state - np.random.uniform(0.1, 0.3), 0.0)
             personal_value = min(personal_value + np.random.uniform(0.02, 0.05), 1.0)
        elif action_index == 4: # Feedback Reminder
             quality = min(quality + np.random.uniform(0.00, 0.01), 1.0)
             emotion_state = np.clip(emotion_state + np.random.normal(0, 0.03), 0.0, 1.0)
        else:
            print(f"Warning: Unknown action index {action_index}. No specific simulation rule applied.")

        # Simulate time passing and update days_to_deadline and deadline_ratio
        days_passed = random.randint(1, 7)
        days_to_deadline = max(0, days_to_deadline - days_passed)
        initial_days = state.context.get('initial_days_to_deadline', 1)
        deadline_ratio = 1.0 - (days_to_deadline / initial_days)
        deadline_ratio = np.clip(deadline_ratio, 0.0, 1.0)

    print(f"State after update: PV={personal_value:.2f}, QL={quality:.2f}, QT={quantity:.2f}, Steps={step_count}, Deadline={deadline_ratio:.2f}, Days Left={days_to_deadline}")

    # Pass the llm_client through the context if it exists
    llm_client_instance = state.context.get("llm_client")
    if llm_client_instance is not None:
        updated_context['llm_client'] = llm_client_instance


    return {
        "personal_value": personal_value,
        "quality": quality,
        "quantity": quantity,
        "emotion_state": emotion_state,
        "deadline_ratio": deadline_ratio,
        "days_to_deadline": days_to_deadline,
        "context": updated_context,
        "module_response": state.module_response # Pass the LLM response through
    }


def check_exit_node(state: GraphState) -> str:
    """
    Checks if the simulation should exit based on state conditions.
    """
    print("\n--- Executing Check Exit Node ---")
    step_count = state.context.get("step_count", 0)
    personal_value = state.personal_value
    quantity = state.quantity
    days_to_deadline = state.days_to_deadline

    print(f"Checking exit conditions: Step Count = {step_count}, Personal Value = {personal_value:.2f}, Quantity = {quantity:.2f}, Days Left = {days_to_deadline}")

    if step_count >= 10: # Reduced steps for a shorter demo
        print("Exit condition met: Maximum steps reached.")
        return "end"
    elif personal_value >= 0.95:
        print("Exit condition met: Personal value goal reached.")
        return "end"
    elif quantity >= 1.0:
        print("Exit condition met: Quantity goal reached (Thesis complete).")
        return "end"
    elif days_to_deadline <= 0:
        print("Exit condition met: Deadline reached.")
        return "end"
    else:
        print("Exit conditions not met. Continuing.")
        return "policy_node"


# 9. Assemble the LangGraph
# Graph assembly remains the same, the model is passed via config
workflow = StateGraph(GraphState)
workflow.add_node("policy_node", policy_node)
workflow.add_node("llm_node", llm_node)
workflow.add_node("core_update_node", core_update_node)
workflow.set_entry_point("policy_node")
workflow.add_edge("policy_node", "llm_node")
workflow.add_edge("llm_node", "core_update_node")
workflow.add_conditional_edges(
    "core_update_node",
    check_exit_node,
    {
        "end": END,
        "policy_node": "policy_node"
    }
)
app = workflow.compile()


# --- PPO Model Training (using Mock Client) ---
print("\n--- Starting PPO Model Training (using Mock Client) ---")

# Define training parameters
total_timesteps = 100000  # You can adjust this
learning_rate = 3e-4
gamma = 0.99
gae_lambda = 0.95
clip_range = 0.2

# Instantiate the mock client for training
mock_client = MockOpenAIClient()

# Create the training environment using the mock client
train_env = DummyVecEnv([make_env_with_client(mock_client)])

# Define MODEL_PATH (Google Drive)
DRIVE_MODEL_DIR = "/content/drive/MyDrive/Colab_Thesis_Models"
# IMPORTANT: Define the filename WITH the .zip extension
MODEL_FILENAME = "thesis_policy.pt.zip" # Corrected filename
MODEL_PATH = os.path.join(DRIVE_MODEL_DIR, MODEL_FILENAME)

# Create the directory if it doesn't exist
os.makedirs(DRIVE_MODEL_DIR, exist_ok=True)
print(f"Model directory ensured at: {DRIVE_MODEL_DIR}")


# Initialize or load the PPO model for TRAINING from Drive
# This block handles continued learning
model = None
print(f"Training block checking for model at: {MODEL_PATH}") # Debug print
# Check for the existence of the file with the .zip extension
if os.path.exists(MODEL_PATH): # MODEL_PATH now includes .zip
    print(f"Model file found at {MODEL_PATH} for training. Attempting to load.") # Debug print
    try:
        temp_env_instance = ThesisEnv(llm_client=mock_client) # Create temp env with mock client for loading
        temp_env = DummyVecEnv([lambda: temp_env_instance])
        # Load using the path that includes .zip
        model = PPO.load(MODEL_PATH, env=temp_env, device="auto", custom_objects={"StateEncoder": StateEncoder})
        print(f"Existing model loaded from {MODEL_PATH}. Continuing training.")
        temp_env.close()
    except Exception as e:
        print(f"Error loading existing model for training from {MODEL_PATH}: {e}. Initializing a new model.")
        model = PPO(
            "MlpPolicy",
            train_env,
            learning_rate=learning_rate,
            gamma=gamma,
            gae_lambda=gae_lambda,
            clip_range=clip_range,
            policy_kwargs=policy_kwargs,
            verbose=1,
            device="auto"
        )
else:
    print(f"No existing model found at {MODEL_PATH}. Initializing a new model.")
    model = PPO(
        "MlpPolicy",
        train_env,
        learning_rate=learning_rate,
        gamma=gamma,
        gae_lambda=gae_lambda,
        clip_range=clip_range,
        policy_kwargs=policy_kwargs,
        verbose=1,
        device="auto"
    )

# Train the PPO model
print(f"Training for {total_timesteps} timesteps...")
model.learn(total_timesteps=total_timesteps)
print("Training finished.")

# Save the trained model to Drive using the path that includes .zip
model.save(MODEL_PATH) # model.save will overwrite if exists
print(f"Trained model saved to {MODEL_PATH}")

# Add a small delay after saving to allow file system to sync, especially with Drive
# This is a potential workaround for timing issues in single-cell execution
print("Adding a small delay after saving...")
time.sleep(5) # Wait for 5 seconds
print("Delay finished.")

# Close the training environment
train_env.close()
print("Training environment closed.")





--- Starting PPO Model Training (using Mock Client) ---
Model directory ensured at: /content/drive/MyDrive/Colab_Thesis_Models
Training block checking for model at: /content/drive/MyDrive/Colab_Thesis_Models/thesis_policy.pt.zip
Model file found at /content/drive/MyDrive/Colab_Thesis_Models/thesis_policy.pt.zip for training. Attempting to load.
Existing model loaded from /content/drive/MyDrive/Colab_Thesis_Models/thesis_policy.pt.zip. Continuing training.
Training for 100000 timesteps...
-----------------------------
| time/              |      |
|    fps             | 940  |
|    iterations      | 1    |
|    time_elapsed    | 2    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 566         |
|    iterations           | 2           |
|    time_elapsed         | 7           |
|    total_timesteps      | 4096        |
| train/                  |             |
| 

In [79]:
# Import the OpenAI library
from openai import OpenAI
# Used to securely store your API key - uncomment if using Colab Secrets
from google.colab import userdata

# Get your OpenAI API key securely
# Or if using Colab Secrets:
openai_api_key_secure = userdata.get('OPENAI_API_KEY')
openai_organization = userdata.get('OPENAI_ORGANIZATION')
openai_project = userdata.get('OPENAI_PROJECT_ID')

# Set your project API key
OpenAI.api_key = openai_api_key_secure
# You must also set organization and project ID
OpenAI.organization = openai_organization
OpenAI.project = openai_project

# Create the OpenAI client
client = OpenAI(api_key= OpenAI.api_key)

print("OpenAI client initialized.")
# Test the client connection (optional)
try:
    # Use the globally available client object
    response = client.chat.completions.create(
      model="gpt-4o-mini", # Using a suitable model for testing
      messages=[{"role": "user", "content": "Hello"}],
      max_tokens=10
    )
    print(f"OpenAI client connected successfully. Response to 'Hallo!': {response.choices[0].message.content}")
except Exception as e:
    print(f"Error connecting to OpenAI client: {e}")
# --- LangGraph Simulation/Inference (using Trained Policy and Real Client) ---
print("\n--- Starting LangGraph Simulation with Trained Policy and Real Client ---")

# Load the trained model for INFERENCE *once* before the stream
# This model object will be passed to the policy_node via the config argument of app.stream
inference_model = None
inference_model_path = MODEL_PATH # Use the Drive path with .zip

print(f"Simulation block checking for model at: {inference_model_path}") # Debug print
if os.path.exists(inference_model_path): # Check for the existence of the file
    print(f"Model file found at {inference_model_path} for simulation. Attempting to load.") # Debug print
    try:
        # Load with a minimal env instance to ensure observation/action space is correct
        temp_env_instance = ThesisEnv() # Use default init (real client implicitly)
        temp_env = DummyVecEnv([lambda: temp_env_instance])
        # Load using the path that includes .zip
        inference_model = PPO.load(inference_model_path, env=temp_env, device="auto", custom_objects={"StateEncoder": StateEncoder})
        print("Trained model loaded successfully for inference.")
        temp_env.close()
    except Exception as e:
        print(f"Error loading trained model for inference from {inference_model_path}: {e}. Simulation will use a random policy.")
        inference_model = None
else:
     print(f"Trained model not found at {inference_model_path} for inference. Simulation will use a random policy.")
     inference_model = None


# Initialize initial_state for simulation
initial_days = random.randint(100, 500) # Simulate a deadline range for initial state
initial_state = GraphState(
    user_input="Start thesis writing process.",
    request_type="initial_request",
    thesis_stage="planning",
    context={"step_count": 0, "initial_days_to_deadline": initial_days}, # Store initial days in context
    personal_value=np.random.uniform(0.4, 0.6),
    quality=np.random.uniform(0.4, 0.6),
    quantity=0.0,
    emotion_state=np.random.uniform(0.4, 0.6),
    deadline_ratio=0.0,
    # Initialize new user/thesis fields
    thesis_field=random.choice(["Computer Science", "Psychology", "History"]),
    procrastination_level=random.choice(["low", "medium", "high"]),
    writing_style=random.choice(["concise", "descriptive", "neutral"]),
    days_to_deadline=initial_days
)

# Pass the global real client to the initial state context for the LLM node
global client # Ensure global client is accessible
initial_state.context["llm_client"] = client

# Pass the loaded inference_model via the 'config' argument of app.stream
# It's accessed via config.get("configurable", {}).get("ppo_model") in the node
# This reverts the policy node back to not loading internally
simulation_config = {"recursion_limit": 500, "configurable": {"ppo_model": inference_model}}


try:
    max_simulation_steps = 1
    step_count = 0
    current_state_dict = initial_state.model_dump()

    print("Streaming graph execution updates:")
    for step_update in app.stream(initial_state, simulation_config): # Pass the config here
        for node_name, update_data in step_update.items():
            # Update current_state_dict by merging updates
            for key, value in update_data.items():
                 if key == 'context':
                      current_state_dict['context'].update(value)
                 else:
                      current_state_dict[key] = value

            if node_name == "core_update_node":
                 step_count = current_state_dict['context'].get('step_count', 0)
                 print(f"\n--- Step {step_count} ---")
                 print(f"State: PV={current_state_dict['personal_value']:.2f}, QL={current_state_dict['quality']:.2f}, QT={current_state_dict['quantity']:.2f}, Deadline={current_state_dict['deadline_ratio']:.2f}, Days Left={current_state_dict['days_to_deadline']}")
                 print(f"Advisor Suggestion: {current_state_dict.get('module_response', 'N/A')}")

        if step_count >= max_simulation_steps:
            print(f"\n--- Reached maximum simulation steps ({max_simulation_steps}) ---")
            break

    print("\n--- Simulation Finished ---")
    final_state_dict = current_state_dict
    print(f"Final State: PV={final_state_dict['personal_value']:.2f}, QL={final_state_dict['quality']:.2f}, QT={final_state_dict['quantity']:.2f}, Steps={final_state_dict['context'].get('step_count', 'N/A')}, Deadline={final_state_dict['deadline_ratio']:.2f}, Days Left={final_state_dict['days_to_deadline']}")

except Exception as e:
    print(f"\n--- Simulation Ended Due to Error ---")
    print(f"Error: {e}")

print("\n--- Combined Cell Execution Complete ---")

OpenAI client initialized.
OpenAI client connected successfully. Response to 'Hallo!': Hello! How can I assist you today?

--- Starting LangGraph Simulation with Trained Policy and Real Client ---
Simulation block checking for model at: /content/drive/MyDrive/Colab_Thesis_Models/thesis_policy.pt.zip
Model file found at /content/drive/MyDrive/Colab_Thesis_Models/thesis_policy.pt.zip for simulation. Attempting to load.
Trained model loaded successfully for inference.
Streaming graph execution updates:

--- Executing Policy Node ---
PPO model found in config. Proceeding with prediction.
Policy Node Output: Recommended Action Index = 4 (Advisor feedback reminder)

--- Executing LLM Abstraction Node (using OpenAI) ---
Using client from state context for LLM node (likely mock).

--- Executing Core Update Node in 'simulate' mode ---
State after update: PV=0.60, QL=0.56, QT=0.00, Steps=1, Deadline=0.00, Days Left=305

--- Executing Check Exit Node ---
Checking exit conditions: Step Count = 1, 